In [1]:
from stable_baselines3 import PPO
import globals as gl
import classes as cl
import time
import os
from environment import PricingGame

In [2]:
gl.initialize()

model = PPO
model_name="PPO_onehot"
timesteps= 500_000
num_timesteps=20
state_onehot=True
costs=[gl.LOW_COST, gl.HIGH_COST]
adv_mixed_strategy = cl.MixedStrategy(strategiesList=[cl.Strategy(
        cl.StrategyType.static, NNorFunc=cl.myopic, name="myopic")], probablitiesArray=[1])

In [3]:
lrs=[0.00008, 0.0003, 0.0009]

In [4]:
for lr in lrs:
    for _ in range(2):
        
        iter_name = f"{model_name}-{str(int(time.time()))}"
        models_dir = os.path.join("models", iter_name)
        log_dir = os.path.join("logs", iter_name)
        if not os.path.exists(models_dir):
            os.makedirs(models_dir)

        if not os.path.exists(log_dir):
            os.makedirs(log_dir)

        env = PricingGame(tuple_costs=costs, adversary_mixed_strategy=adv_mixed_strategy, state_onehot=state_onehot)
        env.reset()

        lr_=(gl.LR if (lr is None) else lr)

        model_ = model('MlpPolicy', env,learning_rate=lr_,verbose=1, tensorboard_log=log_dir, gamma=gl.GAMMA)

        for i in range(num_timesteps):
            model_.learn(total_timesteps=timesteps,
                         reset_num_timesteps=False, tb_log_name=iter_name)
            model_.save(os.path.join(models_dir, str(timesteps*i)))

        # test and write results
        for iter in range(gl.NUM_STOCHASTIC_ITER):
            obs = env.reset()
            done = False

            actions = []
            while not done:
                action, _states = model_.predict(obs)
                obs, reward, done, info = env.step(action)

                actions.append(int(action))
            #   name	ep	costs	adversary	agent_return	adv_return	agent_rewards	actions	agent_prices	adv_prices	agent_demands	adv_demands	    lr	hist	 total_stages	action_step	num_actions  gamma  state_onehot"
            data=[iter_name, timesteps*num_timesteps,("L" if (costs[0]<costs[1]) else "H"), env.adversary_strategy.name, sum(env.profit[0]), sum(env.profit[1]),            str(env.profit[0]), str(actions), str(env.prices[0]), str(env.prices[1]), str(env.demand_potential[0]),str(env.demand_potential[1]), lr_, gl.NUM_ADV_HISTORY, gl.TOTAL_STAGES, gl.ACTION_STEP, gl.NUM_ACTIONS, gl.GAMMA, env.state_onehot]
            cl.write_to_excel(data)


Using cpu device


C:\Users\sjaha\Documents\Anaconda\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


AttributeError: 'PricingGame' object has no attribute 'render_mode'